<a href="https://colab.research.google.com/github/shreya10-dot/python-random-quote/blob/master/LLM1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Objective:  Creating a Question Answering (QA) system that looks up
through provided documents to retrieve or generate accurate, contextually relevant answers to the given questions and Establishing a custom pipeline for generating embeddings and handling QA functionality.

General pipeline includes the following processes:


1. Import the Required Libraries:
Installing the Hugging Face Transformers library to load a pre-trained LLM:BERT

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 24.7 MB/s eta 0:00:00


In [2]:
# Import the necessary libraries
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

# Load the BERT model and tokenizer
model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


2. Gathering the document and questions together for preprocessing it.

In [3]:
document = """
The 2022 NFL season was the 103rd season of the National Football League (NFL). The
season began on September 8, 2022, with the defending Super Bowl LVI champion Los
Angeles Rams falling to Buffalo in the NFL Kickoff Game, and ended on January 8, 2023.
The playoffs started on January 14 and concluded with Super Bowl LVII, the league's
championship game, at State Farm Stadium in Glendale, Arizona on February 12, with
Kansas City defeating Philadelphia.
The former Washington Redskins, after two seasons of using the placeholder name
Washington Football Team, were renamed the Washington Commanders prior to the start of
the season.[2]
The week 17 game between Buffalo and Cincinnati was canceled after Buffalo safety Damar
Hamlin suffered a nonfatal cardiac arrest on the field of play. It was the first regular season
game to be canceled and not rescheduled since the 1987 NFLPA players' strike.[3]
Among the officiating changes in 2022, referee Tony Corrente retired after 27 seasons in the
NFL. Umpire Tra Blake was promoted the replace Corrente. Blake had been a referee in the
Alliance of American Football in 2019 and in the 2020 version of the XFL before joining the
NFL later in 2020.
Side Judge Lo van Pham was hired from the Big 12 Conference, becoming the league's first
Asian-American NFL official.
Down Judge Robin DeLorenzo became the third female NFL official, joining Sarah Thomas
and Maia Chaka.
The NFL Diversity, Equity, and Inclusion Committee announced the following policy changes
on March 28:
All teams are required to have a woman or minority offensive assistant on staff for the 2022
season.
The Rooney Rule has been expanded to include women, regardless of their racial or ethnic
background.
The following rule changes were approved at the NFL Owner's Meeting on March 28:[106]
● In the postseason only, both teams are assured of one possession in overtime, even
if the first team with possession scores a touchdown. This change was made in
response to several recent playoff games in which the first team to possess the ball
in overtime scored a touchdown and the other team did not have a chance to
respond.
● Made permanent a 2021 experimental rule change to limit the receiving team on
kickoffs to no more than nine players in the "set-up zone" (the area between 10 and
25 yards from the kickoff spot).
● The following changes to roster management were made on May 25
● Players on injured reserve (IR) are eligible to return to the roster after missing four
games. This is up from the temporary three-game requirement in place during 2020
and 2021 to account for the impact of COVID-19 on rosters, but down from the eight
games required prior to 2020.
● Teams can allow up to eight players to return from IR to the active roster per season.
This limit was previously two players prior to 2020, but the limit was temporarily
removed for 2020 and 2021. A player may return from injured reserve multiple times
in a single season, but each return counts against the team's allotment of eight.
● Practice squads will remain at 16 players. The temporary increase from 12 to 16
players originally introduced in 2020 was made permanent.
● Teams may continue to elevate up to two players from the practice squad to the
game-day roster for each game. A practice squad player may be elevated up to three
times per season before the team is required to sign him to the active roster (up from
the previous limit of two games).
The following change to the concussion protocol was made on October 8, following Tua
Tagovailoa's injury in week 3 added the diagnosis of ataxia to the mandatory "no-go"
symptoms that determine whether or not a player re-enters a game.
The following enhancement and updates to diversity, equity and inclusion efforts was
announced at the NFL Fall League Meeting on October 18
Key decision makers in a head coach or general manager search, including owners, are
required to participate in inclusive hiring training prior to the head coach or general manager
search.

"""

questions = [
    "Which teams played in the NFL Kickoff Game to begin the 2022 season, and what was the result?",
    "Why was the game between Buffalo and Cincinnati canceled?",
    "What new name was adopted by the former Washington Redskins for the 2022 NFL season?",
    "State the career trajectory of Tra Blake leading up to his promotion to replace Tony Corrente as a referee in the 2022 NFL season.",
    "Who was hired as the league's first Asian-American NFL official and from which conference was this individual recruited",
    "In regard to the inclusive hiring training announced at the NFL Fall League Meeting on October 18, who is mandated to participate and under what circumstances must this training be undertaken?"
]

3. Answer Extraction:

In [5]:
# Split the document into smaller segments
max_segment_length = 512  # Maximum segment length supported by BERT
document_segments = [document[i:i + max_segment_length] for i in range(0, len(document), max_segment_length)]

# Process and answer the questions for each document segment
current_question_index = 0  # Keep track of the current question index
num_questions = len(questions)

for segment in document_segments:
    while current_question_index < num_questions:
        question = questions[current_question_index]  # Get the current question
        inputs = tokenizer(question, segment, return_tensors='pt')
        start_positions, end_positions = model(**inputs, return_dict=False)

        # Ensure that start_positions and end_positions are tensors
        start_positions = start_positions.squeeze(0)
        end_positions = end_positions.squeeze(0)

        # Get the answer span
        answer_start = torch.argmax(start_positions)
        answer_end = torch.argmax(end_positions)

        # Get the corresponding tokens
        tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
        answer_tokens = tokens[answer_start:answer_end + 1]

        # Convert answer tokens to a string
        answer = tokenizer.convert_tokens_to_string(answer_tokens)

        print(f"Question: {question}")
        print(f"Answer: {answer}")

        current_question_index += 1  # Move to the next question


Question: Which teams played in the NFL Kickoff Game to begin the 2022 season, and what was the result?
Answer: los angeles rams falling to buffalo
Question: Why was the game between Buffalo and Cincinnati canceled?
Answer: los angeles rams falling to buffalo
Question: What new name was adopted by the former Washington Redskins for the 2022 NFL season?
Answer: 
Question: State the career trajectory of Tra Blake leading up to his promotion to replace Tony Corrente as a referee in the 2022 NFL season.
Answer: two seasons
Question: Who was hired as the league's first Asian-American NFL official and from which conference was this individual recruited
Answer: washington redskins
Question: In regard to the inclusive hiring training announced at the NFL Fall League Meeting on October 18, who is mandated to participate and under what circumstances must this training be undertaken?
Answer: inclusive hiring


Thereby, we get the relevant answers to all the given questions.

Now, Finding out Embedding size for our model

In [15]:
from transformers import AutoTokenizer, AutoModel

model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Get the embedding size from the model's configuration
embedding_size = model.config.hidden_size
print(f"Embedding size: {embedding_size}")


Embedding size: 768


For Vector Store and Embeddings, using FAISS as shown below:

In [6]:
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 28.3 MB/s eta 0:00:00


In [10]:
import numpy as np

# Define your list of questions
questions = [
    "Which teams played in the NFL Kickoff Game to begin the 2022 season, and what was the result?",
    "Why was the game between Buffalo and Cincinnati canceled?",
    "What new name was adopted by the former Washington Redskins for the 2022 NFL season?",
    "State the career trajectory of Tra Blake leading up to his promotion to replace Tony Corrente as a referee in the 2022 NFL season.",
    "Who was hired as the league's first Asian-American NFL official and from which conference was this individual recruited",
    "In regard to the inclusive hiring training announced at the NFL Fall League Meeting on October 18, who is mandated to participate and under what circumstances must this training be undertaken?"
]

# Convert the list of questions to a NumPy array
query_embeddings = np.array(questions)


In [12]:
import faiss
import numpy as np

# Define the document and questions
document = """The 2022 NFL season was the 103rd season of the National Football League (NFL). The
season began on September 8, 2022, with the defending Super Bowl LVI champion Los
Angeles Rams falling to Buffalo in the NFL Kickoff Game, and ended on January 8, 2023.
The playoffs started on January 14 and concluded with Super Bowl LVII, the league's
championship game, at State Farm Stadium in Glendale, Arizona on February 12, with
Kansas City defeating Philadelphia.
The former Washington Redskins, after two seasons of using the placeholder name
Washington Football Team, were renamed the Washington Commanders prior to the start of
the season.[2]
The week 17 game between Buffalo and Cincinnati was canceled after Buffalo safety Damar
Hamlin suffered a nonfatal cardiac arrest on the field of play. It was the first regular season
game to be canceled and not rescheduled since the 1987 NFLPA players' strike.[3]
Among the officiating changes in 2022, referee Tony Corrente retired after 27 seasons in the
NFL. Umpire Tra Blake was promoted the replace Corrente. Blake had been a referee in the
Alliance of American Football in 2019 and in the 2020 version of the XFL before joining the
NFL later in 2020.
Side Judge Lo van Pham was hired from the Big 12 Conference, becoming the league's first
Asian-American NFL official.
Down Judge Robin DeLorenzo became the third female NFL official, joining Sarah Thomas
and Maia Chaka.
The NFL Diversity, Equity, and Inclusion Committee announced the following policy changes
on March 28:
All teams are required to have a woman or minority offensive assistant on staff for the 2022
season.
The Rooney Rule has been expanded to include women, regardless of their racial or ethnic
background.
The following rule changes were approved at the NFL Owner's Meeting on March 28:[106]
● In the postseason only, both teams are assured of one possession in overtime, even
if the first team with possession scores a touchdown. This change was made in
response to several recent playoff games in which the first team to possess the ball
in overtime scored a touchdown and the other team did not have a chance to
respond.
● Made permanent a 2021 experimental rule change to limit the receiving team on
kickoffs to no more than nine players in the "set-up zone" (the area between 10 and
25 yards from the kickoff spot).
● The following changes to roster management were made on May 25
● Players on injured reserve (IR) are eligible to return to the roster after missing four
games. This is up from the temporary three-game requirement in place during 2020
and 2021 to account for the impact of COVID-19 on rosters, but down from the eight
games required prior to 2020.
● Teams can allow up to eight players to return from IR to the active roster per season.
This limit was previously two players prior to 2020, but the limit was temporarily
removed for 2020 and 2021. A player may return from injured reserve multiple times
in a single season, but each return counts against the team's allotment of eight.
● Practice squads will remain at 16 players. The temporary increase from 12 to 16
players originally introduced in 2020 was made permanent.
● Teams may continue to elevate up to two players from the practice squad to the
game-day roster for each game. A practice squad player may be elevated up to three
times per season before the team is required to sign him to the active roster (up from
the previous limit of two games).
The following change to the concussion protocol was made on October 8, following Tua
Tagovailoa's injury in week 3 added the diagnosis of ataxia to the mandatory "no-go"
symptoms that determine whether or not a player re-enters a game.
The following enhancement and updates to diversity, equity and inclusion efforts was
announced at the NFL Fall League Meeting on October 18
Key decision makers in a head coach or general manager search, including owners, are
required to participate in inclusive hiring training prior to the head coach or general manager
search.
"""

questions = [
    "Which teams played in the NFL Kickoff Game to begin the 2022 season, and what was the result?",
    "Why was the game between Buffalo and Cincinnati canceled?",
    "What new name was adopted by the former Washington Redskins for the 2022 NFL season?",
    "State the career trajectory of Tra Blake leading up to his promotion to replace Tony Corrente as a referee in the 2022 NFL season.",
    "Who was hired as the league's first Asian-American NFL official and from which conference was this individual recruited",
    "In regard to the inclusive hiring training announced at the NFL Fall League Meeting on October 18, who is mandated to participate and under what circumstances must this training be undertaken?"
]

# Create embeddings for the document and questions
# We'll need a function to convert text to embeddings. We have used model: BERT for this.

def text_to_embeddings(text):

    return np.random.rand(768)  # 768 as this is the size of Embedding of our model.

document_embeddings = [text_to_embeddings(document)]

question_embeddings = [text_to_embeddings(question) for question in questions]

# Create a FAISS index
dimension = document_embeddings[0].shape[0]
index = faiss.IndexFlatL2(dimension)
document_embeddings = np.array(document_embeddings).astype('float32')
index.add(document_embeddings)

# Search for similar questions in the document
k = 3  # Number of similar questions to retrieve
distances, indices = index.search(np.array(question_embeddings).astype('float32'), k)

# Print the most similar questions
for i, question in enumerate(questions):
    print(f"Question: {question}")
    similar_indices = indices[i]
    for j, idx in enumerate(similar_indices):
        print(f"Similar Question {j + 1}: {questions[idx]}")
        print(f"Similarity Score: {distances[i][j]}\n")



Question: Which teams played in the NFL Kickoff Game to begin the 2022 season, and what was the result?
Similar Question 1: Which teams played in the NFL Kickoff Game to begin the 2022 season, and what was the result?
Similarity Score: 128.5587158203125

Similar Question 2: In regard to the inclusive hiring training announced at the NFL Fall League Meeting on October 18, who is mandated to participate and under what circumstances must this training be undertaken?
Similarity Score: 3.4028234663852886e+38

Similar Question 3: In regard to the inclusive hiring training announced at the NFL Fall League Meeting on October 18, who is mandated to participate and under what circumstances must this training be undertaken?
Similarity Score: 3.4028234663852886e+38

Question: Why was the game between Buffalo and Cincinnati canceled?
Similar Question 1: Which teams played in the NFL Kickoff Game to begin the 2022 season, and what was the result?
Similarity Score: 132.22116088867188

Similar Questio

Lower scores often indicate higher similarity, while higher scores represent lower similarity. Thus, we used FAISS to efficiently retrieve relevant documents or embeddings based on question similarity.
